Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils

import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
optimal = pd.read_pickle(f"{path}output/pickle_results/real_data/optimal_input_size.pkl")
optimal.head()

,dataset,nb_genes
0,Quake_Smart-seq2_Trachea,500
1,Quake_Smart-seq2_Diaphragm,1000
2,Quake_10x_Spleen,500
3,Young,1000
4,mouse_ES_cell,1000


In [4]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [5]:
df = pd.DataFrame()
lr =0.4
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = optimal[optimal["dataset"] == dataset]["nb_genes"].values[0]
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([0.07]):
        for dropout in [ 0.9
                       ]:
            for noise in [0, 0.01, 0.1, 1
                      ]:
                for layers in [
                    [200, 40, 60]
                              ]:
                    for run in range(3):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=30,
                                             lr=lr,
                                             noise = noise,
                                             temperature=temperature,
                                             dropout=dropout,
                                             
                                             
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["noise"] = noise
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
                        print(f"# {temperature}, {dropout}, {lr}, {layers}", 
                              dresults.get('COMBINED_kmeans_ari', ""),
                              dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
                              dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_noise.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8258284521205065
(1350, 23341) (1350, 23341) keeping 500 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1000 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Quake_10x_Spleen
SCZI  0.9207243162607069
(9552, 23341) (9552, 23341) keeping 500 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Young
SCZI  0.6966379900869195
(5685, 33658) (5685, 33658) keeping 1000 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data mouse_ES_cell
SCZI  0.8004502714264402
(2717, 24175) (2717, 24175) keeping 1000 genes


...

.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Romanov
SCZI  0.7211879386643121
(2881, 21143) (2881, 21143) keeping 500 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Quake_Smart-seq2_Limb_Muscle
SCZI  0.9738741466572893
(1090, 23341) (1090, 23341) keeping 1000 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data Muraro
SCZI  0.7263895607362582
(2122, 19046) (2122, 19046) keeping 1000 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    

>>>>> Data 10X_PBMC
SCZI  0.5902722091957062
(4271, 16653) (4271, 16653) keeping 500 genes


.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    
.# 0.07, 0.9, 0.4, [200, 40, 60]    



In [ ]:
df

In [ ]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

In [ ]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_noise.pkl")
df = df.fillna(0)
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "noise"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).T

In [ ]:
# df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "noise"]).max()

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot( data = comb, y="value", x = "variable")
plt.xticks(rotation = 90)

In [ ]:
sns.barplot(x = "dataset", data = comb, y="value", hue = "variable")
plt.xticks(rotation = 90)

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')